In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from models.managers.db_manager import connect_to_bdd
from src.data.utils.constants import id_to_party_dict

conn = connect_to_bdd()

# Requête pour election_2022_t1 et sauvegarde dans df_election
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Requête pour economie et sauvegarde dans df_economie
query_economie = """
    SELECT avg_1982, avg_1983, avg_1984, avg_1985, avg_1986, avg_1987, avg_1988, avg_1989,
           avg_1990, avg_1991, avg_1992, avg_1993, avg_1994, avg_1995, avg_1996, avg_1997,
           avg_1998, avg_1999, avg_2000, avg_2001, avg_2002, avg_2003, avg_2004, avg_2005,
           avg_2006, avg_2007, avg_2008, avg_2009, avg_2010, avg_2011, avg_2012, avg_2013,
           avg_2014, avg_2015, avg_2016, avg_2017, avg_2018, avg_2019, avg_2020, avg_2021,
           avg_2022, avg_2023, code_postal
    FROM economie
"""
df_economie = pd.read_sql(query_economie, conn)

# Requête pour securite et sauvegarde dans df_securite
query_securite = """
    SELECT code_postal, tauxpourcent
    FROM securite
"""
df_securite = pd.read_sql(query_securite, conn)

# Requête pour densite et sauvegarde dans df_densite
query_densite = """
    SELECT code_postal, dens_pop
    FROM demographie
"""
df_densite = pd.read_sql(query_densite, conn)

# Fermeture de la connexion
conn.close()

# Calculer la moyenne de densité de population pour chaque département
df_densite = df_densite.groupby('code_postal')['dens_pop'].mean().reset_index()

# Calculer la moyenne du taux de pourcentage de sécurité pour chaque département
df_securite = df_securite.groupby('code_postal')['tauxpourcent'].mean().reset_index()

# Mapper les noms des partis politiques aux identifiants numériques
party_to_id_dict = {v: k for k, v in id_to_party_dict.items()}
df_election["winner"] = df_election["winner"].map(party_to_id_dict)

# Convertir la colonne "winner" en float
df_election["winner"] = df_election["winner"].astype(float)

label_encoder = LabelEncoder()

# Merge df_election et df_economie sur la colonne code_postal
df_merged = pd.merge(df_election, df_economie, on="code_postal", how="left")

# Merge df_merged et df_securite sur la colonne code_postal
df_merged = pd.merge(df_merged, df_securite, on="code_postal", how="left")

# Merge df_merged et df_densite sur la colonne code_postal
df_merged = pd.merge(df_merged, df_densite, on="code_postal", how="left")

# Supprimer les lignes contenant des valeurs NaN de df_merged
df_merged.dropna(inplace=True)

# Exclure les données du département de l'Yonne (89) du jeu d'entraînement
df_train = df_merged[df_merged["code_postal"] != 89]

# Sélectionner uniquement les données du département de l'Yonne (89) pour les tests
df_test_yonne = df_merged[df_merged["code_postal"] == 89]

# Ajouter 10 autres départements au département de l'Yonne (89) pour les tests
other_departments = [90, 91, 92, 93, 94, 95, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
for department in other_departments:
    df_test_yonne = pd.concat([df_test_yonne, df_merged[df_merged["code_postal"] == department]])

# Exclure les données des départements ajoutés des données d'entraînement
df_train = df_train[~df_train["code_postal"].isin(other_departments)]

# Séparation des features et de la target pour l'ensemble d'entraînement
X_train = df_train.drop(columns=["Libellé de la commune", "winner"])
y_train = df_train["winner"]

# Séparation des features et de la target pour l'ensemble de test de l'Yonne
X_test_yonne = df_test_yonne.drop(columns=["Libellé de la commune", "winner"])
y_test_yonne = df_test_yonne["winner"]

# Entraînement du modèle avec 1000 itérations
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble d'entraînement
y_pred_train = model.predict(X_train)

# Calcul de l'accuracy du modèle
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Précision du modèle sur l'ensemble d'entraînement:", accuracy_train)

# Prédiction sur les données de test de l'Yonne
y_pred_yonne = model.predict(X_test_yonne)

# Calcul de l'accuracy du modèle sur les données de l'Yonne
accuracy_yonne = accuracy_score(y_test_yonne, y_pred_yonne)
print("Précision du modèle sur les données de l'Yonne:", accuracy_yonne)

/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3168/2275153318.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3168/2275153318.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3168/2275153318.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_securite = pd.read_sql(query_securite, conn)
/

Précision du modèle sur l'ensemble d'entraînement: 0.6506358997791486
Précision du modèle sur les données de l'Yonne: 0.6197489539748954


/Users/erwanbuisson/Library/Mobile Documents/com~apple~CloudDocs/Documents/Interpreteur Python/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
